In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [2]:
import numpy as np
import pandas as pd

df = pd.read_csv('C:/ds_work/source_code/Project_DS/data/train.csv')


In [3]:
df.groupby('Destination').count()
df.groupby('HomePlanet').count()
print('Cabin의 개수',len(df.groupby('Cabin').count()))

,PassengerId,HomePlanet,CryoSleep,Cabin,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
Destination,,,,,,,,,,,,,
55 Cancri e,1800,1769,1756,1750,1766,1757,1771,1763,1752,1761,1760,1757,1800
PSO J318.5-22,796,780,777,778,782,774,775,780,778,780,784,779,796
TRAPPIST-1e,5915,5765,5765,5788,5787,5781,5787,5791,5778,5790,5781,5779,5915


,PassengerId,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
HomePlanet,,,,,,,,,,,,,
Earth,4602,4488,4507,4503,4514,4487,4499,4505,4490,4503,4491,4496,4602
Europa,2131,2073,2070,2094,2085,2089,2095,2085,2078,2086,2094,2085,2131
Mars,1759,1716,1722,1717,1716,1716,1723,1721,1723,1723,1720,1718,1759


Cabin의 개수 6560


컬럼
- Destination 목적지 {55 Cancri e, PSO J318.5-22, TRAPPIST-1e}
- HomePlanet  {Earth , Europa, Mars}
- Cabin의 종류 6560
    - Cabin을 앞 대문자만 남기면 의미가 있을 수 있을까?
    - 돈이 많은 사람이 사는 자리 등으로 의미가 있는지확인
    - A,B,C,D,E,F,G,T로 변환 후 다시 숫자로 0~7까지 변환
- True / False 
    - CryoSleep
    - VIP
    - Transported(target)
- 삭제
    - PassengerId
    - Name

In [4]:
# 범주형 데이터 변환
df = pd.read_csv('C:/ds_work/source_code/Project_DS/data/train.csv')

def df_replace(dataframe):
    
    global df_train,df
    df_temp = df
    df = dataframe
    df = df.replace({'55 Cancri e':0,'PSO J318.5-22':1,'TRAPPIST-1e':2})
    df = df.replace({'Earth':0,'Europa':1,'Mars':2})
    df = df.replace({False:0,True:1})

    # Cabin : 번호  
    df['Cabin'] = df['Cabin'].str[0]
    df.insert(4, 'Cabin_num', df_temp['Cabin'].str[2:-2])
    df.insert(5, 'Cabin_port', df_temp['Cabin'].str[-1])
    df = df.replace({'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'T':7})
    df = df.replace({'P':0,'S':1})


    ## 범주형 데이터 확인할 수 있게 dict로 만들어 놓은 것
    Cabin = {'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'T':7}
    Cabin_port = {'P':0, 'S':1} #P : 배의 좌현(port side) S: 배의 우현(starboard side) 
    Destination = {'55 Cancri e': 0, 'PSO J318.5-22': 1,'TRAPPIST-1e':2}
    HomePlanet = {'Earth':0,'Europa':1,'Mars':2}
    CryoSleep = {'False':0,'True':1}
    VIP = {'False':0,'True':1}
    Transported = {'False':0,'True':1}


    df = df.astype({'HomePlanet':'float','CryoSleep':'float','Cabin':'float','Cabin_num':'float','Cabin_port':'float',
    'Destination':'float','Age':'float','VIP':'float','RoomService':'float','FoodCourt':'float'
    ,'ShoppingMall':'float','Spa':'float','VRDeck':'float','Transported':'float'})
    
df_replace(df)
df_train = df

### null값 처리
XGBOOST를 사용해서 pridct값을 null값으로 대체

In [5]:
# df_homeplanet =  df_train중에서 HomePlanet의 null값이 존재하는 열만 추출 
# -> 훈련된 모델의 값으로 다시 채울 예정
df_homeplanet = df_train[df_train['HomePlanet'].isnull()]

#df_train 중에서 HomePlanet의 null값이 존재하는 열만 추출 -> 추후 훈련할 데이터프레임
df_train01 = df_train.dropna(subset=['HomePlanet'])
# df_train
df_train[df_train['HomePlanet'].isnull()]

,PassengerId,HomePlanet,CryoSleep,Cabin,Cabin_num,Cabin_port,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
59,0064_02,NaN,1.0,4.0,3.0,1.0,2.0,33.0,0.0,0.0,0.0,NaN,0.0,0.0,Colatz Keen,1.0
113,0119_01,NaN,0.0,0.0,0.0,0.0,2.0,39.0,0.0,0.0,2344.0,0.0,65.0,6898.0,Batan Coning,0.0
186,0210_01,NaN,1.0,3.0,6.0,0.0,0.0,24.0,0.0,0.0,0.0,NaN,0.0,0.0,Arraid Inicont,1.0
225,0242_01,NaN,0.0,5.0,46.0,1.0,2.0,18.0,0.0,313.0,1.0,691.0,283.0,0.0,Almone Sté,0.0
234,0251_01,NaN,1.0,2.0,11.0,1.0,0.0,54.0,0.0,0.0,0.0,0.0,0.0,0.0,Diphah Amsive,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8515,9084_01,NaN,0.0,4.0,582.0,0.0,2.0,25.0,0.0,1258.0,0.0,22.0,19.0,0.0,Jurs Mone,0.0
8613,9194_01,NaN,0.0,4.0,603.0,1.0,0.0,53.0,0.0,0.0,4017.0,0.0,13.0,3147.0,NaN,0.0
8666,9248_01,NaN,0.0,5.0,1792.0,1.0,0.0,38.0,NaN,28.0,1208.0,973.0,207.0,0.0,Gian Perle,1.0
8674,9257_01,NaN,0.0,5.0,1892.0,0.0,2.0,13.0,0.0,39.0,0.0,1085.0,24.0,0.0,Ties Apple,0.0


In [6]:
# train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_train01.drop(['PassengerId','HomePlanet','Name'],axis=1),df_train01['HomePlanet'], test_size=0.2, random_state=42)

# scaler = StandardScaler().fit(X_train)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)
X_train.shape
y_train.shape
X_test.shape
y_test.shape

(6793, 13)

(6793,)

(1699, 13)

(1699,)

In [7]:
#XGBoost1
import xgboost as xgb
import pandas as pd
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

model = xgb.XGBClassifier(early_stopping_rounds=10)
eval_set = [(X_test, y_test)]
model.fit(X_train, y_train, eval_set=eval_set, verbose=False)#, verbose=True

pred = model.predict(X_test)
acc = accuracy_score(y_test,pred)
acc

# from sklearn.metrics import confusion_matrix
# import scikitplot as skplt

# pred = model.predict(X_test)
# # 실제 , 예측
# cm = confusion_matrix(y_test, pred)
# print('confusion_matrix : \n', cm)

# skplt.metrics.plot_confusion_matrix(y_test,pred,figsize=(8,6))
# plt.show()

# cl_report = metrics.classification_report(y_test,pred)
# print('리포트:\n',cl_report)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=10, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', ...)

0.9405532666274279

In [8]:
# df_01 은 df_train 중에서 HomePlanet의 값이 null인 경우의 데이터프레임
df_homeplanet_null = df_homeplanet.drop('HomePlanet',axis=1)
df_homeplanet_null

,PassengerId,CryoSleep,Cabin,Cabin_num,Cabin_port,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
59,0064_02,1.0,4.0,3.0,1.0,2.0,33.0,0.0,0.0,0.0,NaN,0.0,0.0,Colatz Keen,1.0
113,0119_01,0.0,0.0,0.0,0.0,2.0,39.0,0.0,0.0,2344.0,0.0,65.0,6898.0,Batan Coning,0.0
186,0210_01,1.0,3.0,6.0,0.0,0.0,24.0,0.0,0.0,0.0,NaN,0.0,0.0,Arraid Inicont,1.0
225,0242_01,0.0,5.0,46.0,1.0,2.0,18.0,0.0,313.0,1.0,691.0,283.0,0.0,Almone Sté,0.0
234,0251_01,1.0,2.0,11.0,1.0,0.0,54.0,0.0,0.0,0.0,0.0,0.0,0.0,Diphah Amsive,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8515,9084_01,0.0,4.0,582.0,0.0,2.0,25.0,0.0,1258.0,0.0,22.0,19.0,0.0,Jurs Mone,0.0
8613,9194_01,0.0,4.0,603.0,1.0,0.0,53.0,0.0,0.0,4017.0,0.0,13.0,3147.0,NaN,0.0
8666,9248_01,0.0,5.0,1792.0,1.0,0.0,38.0,NaN,28.0,1208.0,973.0,207.0,0.0,Gian Perle,1.0
8674,9257_01,0.0,5.0,1892.0,0.0,2.0,13.0,0.0,39.0,0.0,1085.0,24.0,0.0,Ties Apple,0.0


In [9]:
# 훈련된 xgboost모델을 사용해서 df_homeplanet_null의 HomePlanet값을 예측
pred = model.predict(df_homeplanet_null.drop(['PassengerId','Name'],axis=1))
pred

array([2, 1, 1, 0, 1, 0, 0, 2, 0, 1, 2, 0, 2, 1, 2, 1, 0, 2, 0, 2, 1, 1,
       2, 0, 0, 2, 1, 1, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 2, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 2, 0, 1, 0, 2, 0, 0, 2, 1, 0, 0, 0, 1, 0, 0, 0, 2,
       2, 1, 0, 1, 0, 0, 1, 0, 2, 0, 0, 1, 0, 1, 2, 2, 0, 2, 2, 0, 0, 0,
       0, 1, 0, 0, 2, 0, 0, 0, 0, 1, 1, 2, 0, 1, 2, 2, 0, 0, 0, 1, 0, 0,
       2, 1, 1, 0, 0, 0, 2, 0, 0, 1, 1, 0, 1, 1, 1, 2, 2, 2, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 2, 2, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 0, 2, 1, 0, 2, 0, 0, 2, 0, 0, 2, 2, 2, 1,
       2, 2, 0], dtype=int64)

In [10]:

# 예측된 값을 다시 df_homeplanet으로 삽입(null값 채우기)
df_homeplanet['HomePlanet'] = pred
df_homeplanet

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_20756\1491181258.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_homeplanet['HomePlanet'] = pred


,PassengerId,HomePlanet,CryoSleep,Cabin,Cabin_num,Cabin_port,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
59,0064_02,2,1.0,4.0,3.0,1.0,2.0,33.0,0.0,0.0,0.0,NaN,0.0,0.0,Colatz Keen,1.0
113,0119_01,1,0.0,0.0,0.0,0.0,2.0,39.0,0.0,0.0,2344.0,0.0,65.0,6898.0,Batan Coning,0.0
186,0210_01,1,1.0,3.0,6.0,0.0,0.0,24.0,0.0,0.0,0.0,NaN,0.0,0.0,Arraid Inicont,1.0
225,0242_01,0,0.0,5.0,46.0,1.0,2.0,18.0,0.0,313.0,1.0,691.0,283.0,0.0,Almone Sté,0.0
234,0251_01,1,1.0,2.0,11.0,1.0,0.0,54.0,0.0,0.0,0.0,0.0,0.0,0.0,Diphah Amsive,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8515,9084_01,2,0.0,4.0,582.0,0.0,2.0,25.0,0.0,1258.0,0.0,22.0,19.0,0.0,Jurs Mone,0.0
8613,9194_01,1,0.0,4.0,603.0,1.0,0.0,53.0,0.0,0.0,4017.0,0.0,13.0,3147.0,NaN,0.0
8666,9248_01,2,0.0,5.0,1792.0,1.0,0.0,38.0,NaN,28.0,1208.0,973.0,207.0,0.0,Gian Perle,1.0
8674,9257_01,2,0.0,5.0,1892.0,0.0,2.0,13.0,0.0,39.0,0.0,1085.0,24.0,0.0,Ties Apple,0.0


In [11]:

# null값을 채운 dataframe과 df_train01 합치기 
df_update = pd.concat([df_train01,df_homeplanet],axis=0)
df_update

,PassengerId,HomePlanet,CryoSleep,Cabin,Cabin_num,Cabin_port,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,1.0,0.0,1.0,0.0,0.0,2.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0.0
1,0002_01,0.0,0.0,5.0,0.0,1.0,2.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1.0
2,0003_01,1.0,0.0,0.0,0.0,1.0,2.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0.0
3,0003_02,1.0,0.0,0.0,0.0,1.0,2.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0.0
4,0004_01,0.0,0.0,5.0,1.0,1.0,2.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8515,9084_01,2.0,0.0,4.0,582.0,0.0,2.0,25.0,0.0,1258.0,0.0,22.0,19.0,0.0,Jurs Mone,0.0
8613,9194_01,1.0,0.0,4.0,603.0,1.0,0.0,53.0,0.0,0.0,4017.0,0.0,13.0,3147.0,NaN,0.0
8666,9248_01,2.0,0.0,5.0,1792.0,1.0,0.0,38.0,NaN,28.0,1208.0,973.0,207.0,0.0,Gian Perle,1.0
8674,9257_01,2.0,0.0,5.0,1892.0,0.0,2.0,13.0,0.0,39.0,0.0,1085.0,24.0,0.0,Ties Apple,0.0


In [12]:
df_update[df_update['HomePlanet'].isnull()]

,PassengerId,HomePlanet,CryoSleep,Cabin,Cabin_num,Cabin_port,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported


In [13]:
df_update.to_csv('C:/ds_work/source_code/Project_DS/data/train_HomePlanetNull_update.csv', index=False)
